# Quadratures

In [1]:
import numpy as np
from scipy import integrate
from scipy.integrate import quad

def f1(x):
    x_log = np.log(x)
    return np.exp(-(x**2)) * (x_log ** 2)

def f2(x):
    denom = (x ** 3) - (2 * x) - 5
    return 1.0 / denom

def f3(x):
    return (x ** 5) * np.exp(-x) * np.sin(x)

def f4(x, y):
    sqr = np.sqrt(x + y)
    denom = sqr * (1 + x + y)

    return 1.0 / (sqr + denom)

def f5(x, y):
    return (x ** 2) + (y ** 2)
    

In [2]:
def sample(f, start_at = 0.0, until = 1.0, step = 0.01):
    x = np.arange(start_at, until, step)
    y = np.empty(x.shape[0])

    for i in range(y.shape[0]):
        y[i] = f(x[i])

    n = x.shape[0]

    ## Return odd number of samples
    if n % 2 == 0:
        x = x[:n - 1]
        y = y[:n - 1]
    
    return x, y, x[0], x[-1]



## Simpsons's rule

In [3]:
def simpson(x, y):
    integral = 0.0
    n = x.shape[0]
    for i in range(0, n - 2, 2):
        h = x[i + 2] - x[i]
        integral += h * (y[i] + 4 * y[i + 1] + y[i + 2]) / 6.0

    return integral

## Veryfing correctness 

Comparing with scipy's simpson and quad - adaptive quadrature.

In [4]:
## Verify correctness

x, y, start, end = sample(f1, start_at = .5, step = 0.05)

mine = simpson(x, y)
scipys = integrate.simps(y, x)
adapt = quad(f1, start, end)[0]

print('Mine:    ' + '{:.5}'.format(mine))
print('Scipy\'s: ' + '{:.5}'.format(scipys))
print('Adaptive: ' + '{:.5}'.format(adapt))
print('-' * 32)


print('Scipy\'s simpson and mine are close: ' + str(np.allclose(mine, scipys)))
print('Absolute difference between quad: ' + str(np.abs(adapt - mine)))

Mine:    0.045575
Scipy's: 0.045575
Adaptive: 0.045573
--------------------------------
Scipy's simpson and mine are close: True
Absolute difference between quad: 2.290973216141634e-06


In [5]:
## Verify correctness

x, y, start, end = sample(f2, start_at = .5, step = 0.05)

mine = simpson(x, y)
scipys = integrate.simps(y, x)
adapt = quad(f2, start, end)[0]

print('Mine:    ' + '{:.5}'.format(mine))
print('Scipy\'s: ' + '{:.5}'.format(scipys))
print('Adaptive: ' + '{:.5}'.format(adapt))
print('-' * 32)


print('Scipy\'s simpson and mine are close: ' + str(np.allclose(mine, scipys)))
print('Absolute difference between quad: ' + str(np.abs(adapt - mine)))

Mine:    -0.066353
Scipy's: -0.066353
Adaptive: -0.066353
--------------------------------
Scipy's simpson and mine are close: True
Absolute difference between quad: 5.958316479093284e-09


In [6]:
## Verify correctness
x, y, start, end = sample(f3, start_at = .5, step = 0.05)

mine = simpson(x, y)
scipys = integrate.simps(y, x)
adapt = quad(f3, start, end)[0]

print('Mine:    ' + '{:.5}'.format(mine))
print('Scipy\'s: ' + '{:.5}'.format(scipys))
print('Adaptive: ' + '{:.5}'.format(adapt))
print('-' * 32)


print('Scipy\'s simpson and mine are close: ' + str(np.allclose(mine, scipys)))
print('Absolute difference between quad: ' + str(np.abs(adapt - mine)))

Mine:    0.02745
Scipy's: 0.02745
Adaptive: 0.02745
--------------------------------
Scipy's simpson and mine are close: True
Absolute difference between quad: 1.0767101298048143e-07



The error for tested functions seems to be around $10^{-6}$, which is suprisingly good result given that adaptive quadrature may have used much more samples. 

## 2D integrals

In [7]:
def trapzeoidal(f, x_min, x_max, y_min, y_max, grid_x, grid_y):
    integral = 0.0
    dx = (x_max - x_min) / grid_x
    dy = (y_max - y_min) / grid_y

    # Inner points
    for j in range(1, grid_y - 1):
        y = y_min + (dy * j)
        for i in range(1, grid_x - 1):
            x = x_min + (dx * i)
            integral += f(x, y)

    # Outer points
    for i in range(1, grid_x - 1):
        x = x_min + (i * dx)
        integral += f(x, y_max) / 2.0
        integral += f(x, y_min) / 2.0

    for j in range(1, grid_y - 1):
        y = y_min + (j * dy)
        integral += f(x_min, y) / 2.0
        integral += f(x_max, y) / 2.0

    # Corner vertices
    integral += f(x_min, y_min) / 4.0
    integral += f(x_min, y_max) / 4.0
    integral += f(x_max, y_min) / 4.0
    integral += f(x_max, y_max) / 4.0

    integral *= dx * dy

    return integral

In [13]:
x_min, x_max = 0.0, 1.0
y_min_f = lambda x: 0 
y_max_f = lambda x: 1 - x

integral = integrate.dblquad(f4, x_min,x_max, y_min_f, y_max_f)[0]
print(f'Library\'s result: ' + '{:.5}'.format(integral))

Library's result: 0.25916


In [11]:
x_min, x_max = -3, 3
y_min, y_max = -5, 5
y_min_f = lambda x: -5
y_max_f = lambda x: 5

for grid_size in [100, 1000, 1500, 2500]:
    mine = trapzeoidal(f5, x_min, x_max, y_min, y_max, grid_size, grid_size)
    scipys = integrate.dblquad(f5, x_min, x_max, y_min_f, y_max_f)[0]
    print(f'Mine with size: {grid_size}x{grid_size}: ' + '{:.5}'.format(mine))
    print('Scipy\'s:               ' + '{:.5}'.format(scipys), end = '\n\n')
    print('Absolute difference: ' + str(np.abs(mine - scipys)))
    print('-' * 32)
    

Mine with size: 100x100: 653.94
Scipy's:               680.0

Absolute difference: 26.06159840000271
--------------------------------
Mine with size: 1000x1000: 677.29
Scipy's:               680.0

Absolute difference: 2.708457671846759
--------------------------------
Mine with size: 1500x1500: 678.19
Scipy's:               680.0

Absolute difference: 1.8082007924542722
--------------------------------
Mine with size: 2500x2500: 678.91
Scipy's:               680.0

Absolute difference: 1.0861515312827805
--------------------------------
